In [2]:
%cd /content/drive/MyDrive/Colab Notebooks/OxHack20
!pip install yolov5
!pip install face_detection

/content/drive/MyDrive/Colab Notebooks/OxHack20
     |████████████████████████████████| 837 kB 5.0 MB/s 
     |████████████████████████████████| 87 kB 6.3 MB/s 
     |████████████████████████████████| 132 kB 46.4 MB/s 
     |████████████████████████████████| 78 kB 6.6 MB/s 
     |████████████████████████████████| 596 kB 39.8 MB/s 
     |████████████████████████████████| 79 kB 6.0 MB/s 
     |████████████████████████████████| 8.6 MB 39.5 MB/s 
     |████████████████████████████████| 138 kB 46.3 MB/s 
     |████████████████████████████████| 127 kB 51.2 MB/s 
     |████████████████████████████████| 60.4 MB 1.2 MB/s 
     |████████████████████████████████| 4.3 MB 40.2 MB/s 
  Created wheel for fire: filename=fire-0.4.0-py2.py3-none-any.whl size=115942 sha256=d9308373b7731f294176fd96693daa6646675ac395a548db27b73496d264d6b1
  Stored in directory: /root/.cache/pip/wheels/8a/67/fb/2e8a12fa16661b9d5af1f654bd199366799740a85c64981226
Successfully built fire
  Attempting uninstall: urllib3
    Fou

  Created wheel for face-detection: filename=face_detection-0.2.2-py3-none-any.whl size=25574 sha256=31068a30d51428ed5338f9eaf3a127d66363566ce5084dbd1dff6a49cc5a18b6
  Stored in directory: /root/.cache/pip/wheels/dd/2d/84/c7a9ce1760b00f8b1d431c9564b108acfb98f588c18d4a1a9c
Successfully built face-detection


This function uses 2 pre-trained models: Yolo version 5 for human object detection, and RetinaNetMobile for face detection. 

The **input** of function is any lecture video, preferable from Oxford Mathematics Institute Lecture Theatre 1. 

The **output** of function is a json file indicating timestamp, detected lecturer, and his/her face:

```
file name: videoname.json
{
  time_stamp: index i frame number
  boss_position: [xmin, xmax, ymin, ymax]
  boss_status: True/False.     True indicate there is face on line
  face_status: (-1,-1,-1,-1) if none, otherwise gives [xmin,xmax,ymin,ymax]
}


```



In [3]:
import yolov5
import cv2
import json
import face_detection
import warnings

def check_inside(box1,box2,epsilon = 3):
  #box in form [x1, x2, y1, y2]
  # return True if box1 totally inside box2
  ax1,ax2,ay1,ay2 = box1
  bx1,bx2,by1,by2 = box2
  return (ax1+3 >= bx1 and ax2 <= bx2+3 and ay1+3>=by1 and ay2<=by2+3)

def transform_face_box(box_pre):
  #box_pre in form # x1, y1, x2, y2,conf
  #return box in form [x1,x2,y1,y2]
  x1, y1, x2, y2,conf = box_pre
  return [x1,x2,y1,y2]

def transform_man_box(box_pre):
  #box_pre in form # x1, y1, x2, y2
  #return box in form [x1,x2,y1,y2]
  x1, y1, x2, y2 = box_pre
  return [x1,x2,y1,y2]

def generate_game_file(video_path):
  # Output a json file in form
  '''
    videoname.json
    {
    time_stamp: index i frame number
    boss_position: [x1, x2, y1, y2]
    boss_status: True/False.     True indicate there is face on line
    face_status: (-1,-1,-1,-1) if none, otherwise gives [x1,x2,y1,y2]
    }

  '''
  # load model of human detection 
  model = yolov5.load('yolov5s.pt')
  detector = face_detection.build_detector(
  "RetinaNetMobileNetV1", confidence_threshold=.5, nms_iou_threshold=.3)
  videoCap = cv2.VideoCapture(video_path)
  fps = 30 #round(videoCap.get(cv2.CAP_PROP_FPS))
  # load the pre-trained model of face detection
  
  resol_width = videoCap.get(cv2.CAP_PROP_FRAME_WIDTH)
  resol_height = cv2.CAP_PROP_FRAME_HEIGHT

  
  # for every one frame
  frame_count = 0
  time_stamp = 0
  jsonResolu = (resol_width,resol_height)
  jsonList = []
  gap = 2
  with open('json_data.json', 'w') as outfile:
    while True:
      ret, img=videoCap.read()
      if ret:
        frame_count+=1
        if (frame_count*gap)%fps == 0:
            # perform inference
            results = model(img)
            # parse results
            predictions = results.pred[0]
            boxes = (predictions[:, :4]).tolist() # x2, x1, y2, y1
            scores = (predictions[:, 4]).tolist()
            categories = (predictions[:, 5]).tolist()
            # Initialize the holder
            man_bbox = [-1,-1,-1,-1]
            face_bbox = [-1,-1,-1,-1]
            face_front = False
            #Extract the prof prosition
            if 0.0 in categories:
              # if we detect the prof
              man_index = scores.index(max([scores[i] for i, e in enumerate(categories) if e == 0.0]))
              man_bbox = transform_man_box(boxes[man_index]) # x1, y1, x2, y2
              
              face_bboxes_pre = detector.detect(img).tolist() 
              face_bboxes = [transform_face_box(item) for item in face_bboxes_pre]
              

              for item in face_bboxes:
                if check_inside(item,man_bbox):
                  face_bbox = item
                  face_front = True
                  break
            # write out the facebbox and the man_bbox
            data = {
                'time_stamp': time_stamp,
                'boss_pos': man_bbox,
                'face_pos': face_bbox,
                'face_front':face_front
            }
            
            jsonList.append(data)
            time_stamp+=1
      else:
        jsonfile = {
            "time_gap_per_stamp": round(1/gap,2),
            "resolution":jsonResolu,
            "data":jsonList
        }
        json_str = json.dumps(jsonfile)
        outfile.write(json_str)
        print("game file generated")
        break
if __name__ == "__main__":
  warnings.filterwarnings("ignore")
  generate_game_file('lec_sample.mp4')

Downloading: "https://folk.ntnu.no/haakohu/RetinaFace_mobilenet025.pth" to /root/.cache/torch/hub/checkpoints/RetinaFace_mobilenet025.pth


  0%|          | 0.00/1.71M [00:00<?, ?B/s]

game file generated
